In [1]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import datetime
import warnings

warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('crime_violence.csv', encoding='cp949', index_col=0)
df.head()

,20년1월_폭력,20년2월_폭력,20년3월_폭력,20년4월_폭력,20년5월_폭력,20년6월_폭력,20년7월_폭력,20년8월_폭력,20년9월_폭력,20년10월_폭력,...,22년3월_폭력,22년4월_폭력,22년5월_폭력,22년6월_폭력,22년7월_폭력,22년8월_폭력,22년9월_폭력,22년10월_폭력,22년11월_폭력,22년12월_폭력
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,339,221,334,324,317,337,295,319,154,270,...,317,287,325,297,335,322,312,324,327,339
강동구,173,145,136,126,154,176,198,176,34,187,...,136,160,159,171,141,154,162,151,197,165
강북구,164,164,177,176,158,153,163,139,102,139,...,129,146,186,197,143,154,152,159,141,129
강서구,215,195,218,195,201,229,225,207,74,220,...,206,229,205,190,242,211,184,226,207,148
관악구,205,157,205,231,232,224,247,2215,154,238,...,162,192,196,230,260,231,206,211,281,232


In [3]:
df.columns

Index(['20년1월_폭력', '20년2월_폭력', '20년3월_폭력', '20년4월_폭력', '20년5월_폭력', '20년6월_폭력',
       '20년7월_폭력', '20년8월_폭력', '20년9월_폭력', '20년10월_폭력', '20년11월_폭력',
       '20년12월_폭력', '21년1월_폭력', '21년2월_폭력', '21년3월_폭력', '21년4월_폭력', '21년5월_폭력',
       '21년6월_폭력', '21년7월_폭력', '21년8월_폭력', '21년9월_폭력', '21년10월_폭력',
       '21년11월_폭력', '21년12월_폭력', '22년1월_폭력', '22년2월_폭력', '22년3월_폭력',
       '22년4월_폭력', '22년5월_폭력', '22년6월_폭력', '22년7월_폭력', '22년8월_폭력', '22년9월_폭력',
       '22년10월_폭력', '22년11월_폭력', '22년12월_폭력'],
      dtype='object')

In [4]:
pattern = ['20년', '21년', '22년']
months = [f'{month}월' for month in range(1, 13)]
sorted_columns = [f'{year}{month}_폭력' for year in pattern for month in months]

# 패턴에 따라 칼럼 이름 정렬
df = df.reindex(columns=sorted_columns)

In [5]:
df.head()

,20년1월_폭력,20년2월_폭력,20년3월_폭력,20년4월_폭력,20년5월_폭력,20년6월_폭력,20년7월_폭력,20년8월_폭력,20년9월_폭력,20년10월_폭력,...,22년3월_폭력,22년4월_폭력,22년5월_폭력,22년6월_폭력,22년7월_폭력,22년8월_폭력,22년9월_폭력,22년10월_폭력,22년11월_폭력,22년12월_폭력
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,339,221,334,324,317,337,295,319,154,270,...,317,287,325,297,335,322,312,324,327,339
강동구,173,145,136,126,154,176,198,176,34,187,...,136,160,159,171,141,154,162,151,197,165
강북구,164,164,177,176,158,153,163,139,102,139,...,129,146,186,197,143,154,152,159,141,129
강서구,215,195,218,195,201,229,225,207,74,220,...,206,229,205,190,242,211,184,226,207,148
관악구,205,157,205,231,232,224,247,2215,154,238,...,162,192,196,230,260,231,206,211,281,232


In [6]:
data = df.T
data.to_csv('시계열분석_전처리_폭력.csv', encoding='cp949')
data.head()

자치구,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
20년1월_폭력,339,173,164,215,205,149,204,134,165,98,...,132,129,240,114,252,149,187,167,178,153
20년2월_폭력,221,145,164,195,157,96,157,146,181,96,...,87,119,171,116,198,114,140,126,116,143
20년3월_폭력,334,136,177,218,205,133,179,130,176,101,...,90,116,223,95,225,142,163,156,137,176
20년4월_폭력,324,126,176,195,231,124,138,110,180,109,...,100,121,244,107,213,146,149,119,96,176
20년5월_폭력,317,154,158,201,232,142,212,142,205,103,...,95,102,247,113,261,147,154,149,126,183


In [96]:
df = pd.read_csv('시계열분석_전처리_폭력.csv', encoding='cp949')
df.head()

,Date,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,20년1월_폭력,339,173,164,215,205,149,204,134,165,...,132,129,240,114,252,149,187,167,178,153
1,20년2월_폭력,221,145,164,195,157,96,157,146,181,...,87,119,171,116,198,114,140,126,116,143
2,20년3월_폭력,334,136,177,218,205,133,179,130,176,...,90,116,223,95,225,142,163,156,137,176
3,20년4월_폭력,324,126,176,195,231,124,138,110,180,...,100,121,244,107,213,146,149,119,96,176
4,20년5월_폭력,317,154,158,201,232,142,212,142,205,...,95,102,247,113,261,147,154,149,126,183


In [97]:
# '범죄건수' 문자열 삭제
df['Date'] = df['Date'].str.replace('_폭력', '')

df['Date'] = df['Date'].str.replace(r'(\d+)년(\d+)월', r'20\1\2', regex=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.strftime('%Y%m')

df.head()

,Date,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,202001,339,173,164,215,205,149,204,134,165,...,132,129,240,114,252,149,187,167,178,153
1,202002,221,145,164,195,157,96,157,146,181,...,87,119,171,116,198,114,140,126,116,143
2,202003,334,136,177,218,205,133,179,130,176,...,90,116,223,95,225,142,163,156,137,176
3,202004,324,126,176,195,231,124,138,110,180,...,100,121,244,107,213,146,149,119,96,176
4,202005,317,154,158,201,232,142,212,142,205,...,95,102,247,113,261,147,154,149,126,183


In [98]:
gu_name = ['종로구', 'Date']

last_3year = list()
for i in range(2023, 2027):
    for j in range(1, 13):
        last_3year.append(['%04d-%02d' % (i,j)])
        
        
last_3year = pd.DataFrame(last_3year, columns = ['ds'])
last_3year['ds']= pd.to_datetime(last_3year['ds'])

for i in range(len(df['Date'])):
    date_time_str = df['Date'][i]
    
    date = datetime.datetime.strptime(str(date_time_str), '%Y%m')  #datetime.datetime.strptime(date, "%Y-%m-%d")
    
    df['Date'][i] = date

gangdong_predict = pd.DataFrame()

for i,name in enumerate(gu_name):
    data = pd.DataFrame()
    data = df[[name,'Date']]
    data.columns = ['y','ds']
    data['ds'] = pd.to_datetime(data['ds'])

    model = Prophet().fit(data)
    print(i,"model_fit")
    forecast = model.predict(last_3year)
    print(i,"model_predict")
    gangdong_predict = pd.concat([gangdong_predict, forecast['yhat']], ignore_index = True, axis=1)

15:51:19 - cmdstanpy - INFO - Chain [1] start processing
15:51:19 - cmdstanpy - INFO - Chain [1] done processing


0 model_fit
0 model_predict


TypeError: Invalid object type at position 0

In [99]:
gangdong_predict = pd.concat([gangdong_predict, forecast['ds']], ignore_index = True, axis=1)

gangdong_predict.columns = ['종로구', 'Date']

gangdong_predict.to_csv('eunpyeong_violence_predict.csv', encoding='cp949',index = False)